Some of the features that has and hasn't been done:

- [X] Display points from all views simultaneously.
- [X] Give color according to vehicle type
- [X] Display the CCTV views
- [X] Display the bounding boxes to CCTV views.

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import time

region_data = pd.read_csv('regions.csv')
region_data

,cctv_folder_parent,map_image_name
0,SIMPANG 3 DPR,SIMPANG 3 DPR.jpeg
1,SIMPANG 4 CUNGKING,SIMPANG 4 CUNGKING.jpeg
2,SIMPANG 4 PATUNG KUDA,SIMPANG 4 PATUNG KUDA.jpeg
3,SIMPANG 5 PUSAT KOTA,SIMPANG 5 BANYUWANGI.jpeg


In [2]:
cctv_data = pd.read_csv('cctv_views.csv')
cctv_data

,region_name,cctv_view_name
0,SIMPANG 3 DPR,dpr selatan
1,SIMPANG 3 DPR,dpr timur
2,SIMPANG 3 DPR,dpr utara
3,SIMPANG 4 PATUNG KUDA,patungkuda barat
4,SIMPANG 4 PATUNG KUDA,patungkuda selatan
5,SIMPANG 4 PATUNG KUDA,patungkuda utara


In [16]:
# Define a CCTV path of image whose region has been detected
# Define the map path as well.

# You can freely change this depending on the condition of your database.
region_name = "SIMPANG 4 PATUNG KUDA"
cctv_range = {'start':0, 'end': 3300}
cctv_time = '1200'

region_cctv_views = cctv_data[cctv_data['region_name'] == region_name]['cctv_view_name'].values.tolist()

region_map_path = region_data[region_data['cctv_folder_parent'] == region_name]
region_map_path = 'map/' + region_map_path['map_image_name'].values.tolist()[0]

region_coords = pd.read_csv('coords/cctv/{:}.csv'.format(region_name))

print("CCTV VIEWS :", region_cctv_views)
print("MAP PATH   :", region_map_path)
print("REGION COORDINATES:")
region_coords

CCTV VIEWS : ['patungkuda barat', 'patungkuda selatan', 'patungkuda utara']
MAP PATH   : map/SIMPANG 4 PATUNG KUDA.jpeg
REGION COORDINATES:


,view,pt1a,pt1b,pt1c,pt1d,pt2a,pt2b,pt2c,pt2d,pt3a,pt3b,pt3c,pt3d
0,patungkuda selatan,97 273,363 236,1074 476,187 770,0 0,1080 0,1080 720,0 720,400 717,295 644,392 513,492 587
1,patungkuda barat,132 283,396 262,1080 505,211 720,0 0,1080 0,1080 720,0 720,364 296,475 236,578 379,458 450
2,patungkuda utara,510 170,668 168,664 437,0 390,0 0,1080 0,1080 720,0 720,648 167,748 239,598 437,497 362


In [4]:
map_image = cv2.imread(region_map_path)

In [5]:
def get_points(pandas_frame, view_name):
    polygons = {}
    row = pandas_frame[pandas_frame['view'] == view_name]
    point_id = ['a', 'b', 'c', 'd']
    polygon_id = ['1', '2', '3']
    for i in polygon_id:
        polygon = []
        for j in point_id:
            point_name = 'pt'+ i + j
            point_array = [int(p) for p in row[point_name].values.tolist()[0].split()]
            polygon.append(point_array)
        polygons[i] = np.float32(polygon)
    return polygons

In [6]:
def warpPerspectivePoints(pts, M):
    n = pts.shape[0]
    trans = np.hstack((pts, np.ones((n,1)))) # Add ones at the bottom of matrix
    trans = np.matmul(M, np.transpose(trans))
    return np.transpose(trans[0:2,:] / trans[2:])

##### Pseudocode:

For every CCTV views:

1. Eliminate non-vehicles point (can be obtained from the "object" header)
   For example, vehicles: truck, car, motorbike
1. For every bounding box vehicles:
   1. Find the bounding box center point
   1. Map the point according to transformation matrix M
1. Now, for displaying purposes:
   1. Draw the map and all the transformed points
   1. In new window, draw the CCTV image with boxes

In [7]:
coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

def eliminate_non_vehicle(series):
    detections = series['object_probs'].split(' ')
    if detections[0] in coco_vehicle_labels:
        return True
    return False

def pause_and_is_finished():
    # Pauses and quit button purposes
    k = cv2.waitKey(25) & 0xFF
    finished = k == ord_q
    if k == ord_p:
        while True:
            k = cv2.waitKey(25) & 0xFF
            finished = k == ord_q
            if k == ord_p or k == ord_q:
                break
    return finished

def generate_equal_rgb_color_distance(n):
    colors = []
    for i in range(n):
        color = np.uint8(cv2.cvtColor(np.float32([[[i*360/n, 0.8, 0.9]]]), cv2.COLOR_HSV2RGB)[0, 0] * 255)
        colors.append(color)
    return colors

coco_colors = generate_equal_rgb_color_distance(len(coco_vehicle_labels))
coco_colors

def get_color_from_class(row):
    class_name = row.split()[0]
    class_index = coco_vehicle_labels.index(class_name)
    return coco_colors[class_index]

In [8]:
# Let's define a dictionary to store variables.
# These variables include precomputed value as well, such as
# the transformation matrix.

objs = {}
for i in range(len(region_cctv_views)):
    obj_dict = {}
    
    rctv = region_cctv_views[i]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))    

    # Get detections CSV file name length
    # It is assumed that all files has the same name length.
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    filename_length = len(file_ext_first_file[0])
    
    # Populate variables to the object dictionary.    
    obj_dict['rctv'] = rctv
    obj_dict['path_cctv_view'] = path_cctv_view
    obj_dict['pts1'] = pts1
    obj_dict['pts2'] = pts2
    obj_dict['filename_length'] = filename_length
    obj_dict['M'] = M
    objs[i] = obj_dict

In [ ]:
ord_q = ord('q') # For quit button
ord_p = ord('p') # For pause button
finished = False
                
# Resize the map image so that it is faster to display
RESIZE_CONSTANT = 0.55
CCTV_VIEW_RESIZE = 0.25

# Other constants
TEXT_POSITION    = np.int32(np.array([25, 40]) * RESIZE_CONSTANT)
TEXT_SIZE        = 1
TEXT_COLOR       = (0, 0, 0)
TEXT_BOLDNESS    = round(2 * RESIZE_CONSTANT)
POINT_SIZE       = round(10 * RESIZE_CONSTANT)
BOX_OUTLINE_SIZE = round(10 * CCTV_VIEW_RESIZE)

map_image_resized = cv2.resize(map_image, (0,0), fx=RESIZE_CONSTANT, fy=RESIZE_CONSTANT)
force_quit = False


# Display the legends
LEGEND_CANVAS_SIZE_X = max([len(c) for c in coco_vehicle_labels] + [len('Legends:')]) * 15 + 20
LEGEND_CANVAS_SIZE_X += 20 # Add 20 px more for color
LEGEND_CANVAS_SIZE_Y = (len(coco_vehicle_labels) + 1) * 35 + 20
LEGEND_FONT_SIZE     = 0.8

legend_canvas = np.uint8(np.ones((LEGEND_CANVAS_SIZE_Y, LEGEND_CANVAS_SIZE_X, 3)) * 255)
cv2.putText(legend_canvas, 'Legends:',
            (15, 30), cv2.FONT_HERSHEY_SIMPLEX, LEGEND_FONT_SIZE,
            (0, 0, 0), 1, cv2.LINE_AA)

for i, label in enumerate(coco_vehicle_labels):
    cv2.putText(legend_canvas, label,
            (45, 35 * (i+1) + 30), cv2.FONT_HERSHEY_SIMPLEX, LEGEND_FONT_SIZE,
            (0, 0, 0), 0, cv2.LINE_AA)
    cv2.circle(legend_canvas, (25, 35 * (i+1) + 15 + 5),
               10, coco_colors[i].tolist(), -1)
    
cv2.imshow('Legend', legend_canvas)

try:
    for i in range(cctv_range['start'], cctv_range['end'] + 1):
        
        # Copy the image. Later we will draw something.
        # We don't want the original to be overwritten.
        new_map_image = map_image_resized.copy()
        image_cctvs = np.array([[[]]])
        
        # Give frame number
        cv2.putText(new_map_image, 'Frame number: {:}'.format(i),
                    tuple(TEXT_POSITION.tolist()),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SIZE * RESIZE_CONSTANT,
                    TEXT_COLOR, TEXT_BOLDNESS, cv2.LINE_AA)
        
        for j in range(len(region_cctv_views)):
            obj = objs[j]
            
            # Print current progress (for debug purposes)
            print('\r' + str(i) + '-' + obj['rctv'] + '   ', end='')
            
            M = obj['M']
            filename_length = obj['filename_length']
            path_cctv_view = obj['path_cctv_view']
            
            csv_file_name = str(i).zfill(filename_length)
            csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)
            
            try: 
                # Read CSV
                objects = pd.read_csv(csv_path)

                # Eliminate non-vehicles object
                n_detection = objects.shape[0]
                if (n_detection > 0):
                    objects = objects[objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)]

                # For every bounding box vehicles, find the bounding box center point
                n_detection = objects.shape[0]
                if (n_detection > 0):
                    # Get colors
                    colors = objects['object_probs'].apply(lambda row: get_color_from_class(row)).values.tolist()
                    centers = objects.apply(lambda r: [
                                      (r['left'] + r['right'])/2,
                                      (r['top'] + r['bottom'])/2
                                      ], axis=1)

                    # Transform the points
                    points_np = np.array([c for c in centers])
                    points_transform = np.float64(warpPerspectivePoints(points_np, M))

                    # Resize the points and the map image so that it is faster to display
                    points_transform *= RESIZE_CONSTANT
                    points_transform = np.int32(points_transform)

                    # Map the points to the image
                    for k, pts_t in enumerate(points_transform):
                        cv2.circle(new_map_image, tuple(pts_t), POINT_SIZE, colors[k].tolist(), -1)
            except Exception:
                print(csv_path + " error.")
                continue
            
            # Read CCTV image, replacing CSV path with JPG path
            cctv_view_image = cv2.imread(csv_path[:-3] + 'jpg')
            cctv_view_image = cv2.resize(cctv_view_image, (0,0), fx=CCTV_VIEW_RESIZE, fy=CCTV_VIEW_RESIZE)
            
            # Draw bounding boxes
            if (n_detection > 0):
                boxes = objects.apply(lambda r: [
                                  [r['left'], r['top']],
                                  [r['right'], r['bottom']]
                                  ], axis=1)
                # Resize the boxes
                boxes = np.int32(np.int32([k for k in boxes.values]) * CCTV_VIEW_RESIZE)
                for k, box in enumerate(boxes):
                    cv2.rectangle(cctv_view_image, tuple(box[0]), tuple(box[1]), tuple(colors[k].tolist()),
                                  BOX_OUTLINE_SIZE)
            
            if (j == 0):
                image_cctvs = cctv_view_image
            else:
                image_cctvs = np.concatenate((image_cctvs, cctv_view_image), axis=0)
        try:
            cv2.imshow('Image', new_map_image)
            cv2.imshow('Image CCTV', image_cctvs)
        except:
            continue
        if pause_and_is_finished():
            force_quit = True
            break
    
    print("\nFinished!")
    if not force_quit:
        cv2.putText(new_map_image, 'Frame number: {:} ~FINISHED!'.format('      '),
                    tuple(TEXT_POSITION.tolist()),
                    cv2.FONT_HERSHEY_SIMPLEX, TEXT_SIZE * RESIZE_CONSTANT,
                    TEXT_COLOR, TEXT_BOLDNESS, cv2.LINE_AA)
        cv2.imshow('Image', new_map_image)
        
        while not pause_and_is_finished():
            time.sleep(0.2)
finally:
    cv2.destroyAllWindows()

8-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000008.csv error.
8-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000008.csv error.
8-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000008.csv error.
9-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000009.csv error.
9-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000009.csv error.
9-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000009.csv error.
11-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000011.csv error.
11-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000011.csv error.
11-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000011.csv error.
16-patungkuda barat   

237-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000237.csv error.
237-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000237.csv error.
237-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000237.csv error.
241-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000241.csv error.
241-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000241.csv error.
241-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000241.csv error.
255-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000255.csv error.
255-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000255.csv error.
255-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000255.csv error.
259-pat

389-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000389.csv error.
389-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000389.csv error.
389-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000389.csv error.
397-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000397.csv error.
397-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000397.csv error.
397-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000397.csv error.
398-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000398.csv error.
398-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000398.csv error.
398-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000398.csv error.
404-pat

535-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000535.csv error.
535-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000535.csv error.
535-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000535.csv error.
537-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000537.csv error.
537-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000537.csv error.
537-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000537.csv error.
547-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000547.csv error.
547-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000547.csv error.
547-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000547.csv error.
554-pat

720-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000720.csv error.
720-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000720.csv error.
720-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000720.csv error.
722-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000722.csv error.
722-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000722.csv error.
722-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000722.csv error.
728-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0000728.csv error.
728-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0000728.csv error.
728-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0000728.csv error.
733-pat

...

1984-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0001984.csv error.
1984-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0001984.csv error.
1984-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0001984.csv error.
1995-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0001995.csv error.
1995-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0001995.csv error.
1995-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0001995.csv error.
2002-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002002.csv error.
2002-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002002.csv error.
2002-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002002.csv error

2173-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002173.csv error.
2173-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002173.csv error.
2173-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002173.csv error.
2178-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002178.csv error.
2178-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002178.csv error.
2178-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002178.csv error.
2182-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002182.csv error.
2182-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002182.csv error.
2182-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002182.csv error

2301-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002301.csv error.
2301-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002301.csv error.
2301-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002301.csv error.
2308-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002308.csv error.
2308-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002308.csv error.
2308-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002308.csv error.
2309-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002309.csv error.
2309-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002309.csv error.
2309-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002309.csv error

2453-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002453.csv error.
2453-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002453.csv error.
2453-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002453.csv error.
2455-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002455.csv error.
2455-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002455.csv error.
2455-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002455.csv error.
2460-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002460.csv error.
2460-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002460.csv error.
2460-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002460.csv error

2614-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002614.csv error.
2614-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002614.csv error.
2614-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002614.csv error.
2636-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002636.csv error.
2636-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002636.csv error.
2636-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002636.csv error.
2641-patungkuda barat   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda barat 1200_x264.mp4//0002641.csv error.
2641-patungkuda selatan   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda selatan 1200_x264.mp4//0002641.csv error.
2641-patungkuda utara   ./frame/cctv_SIMPANG 4 PATUNG KUDA_patungkuda utara 1200_x264.mp4//0002641.csv error

---
##### From this point, the cells below are used for sandbox purposes only.

In [10]:
# Try for single video frames first
try:
    rctv = region_cctv_views[2]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    print("VIEW:", rctv)
    print("PTS 1:", str(pts1).replace("\n", ""))
    print("PTS 2:", str(pts2).replace("\n", ""))

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))
    
    # Draw polygon containing the transformed region
    overlay_transform = np.uint8(np.ones(map_image.shape)) * 255
    cv2.fillPoly(overlay_transform, [np.int32(pts_transform)], (0, 0, 255))
    cv2.fillPoly(map_image_filled, [np.int32(pts_transform)], (255, 0, 0))
    
    print(pts_transform.tolist())

    # Get file name length (it is assumed to be same for all the video frames)
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    print("FIRST FILE:", file_ext_first_file)
    filename_length = len(file_ext_first_file[0])

    coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

    def strip(text):
        try:
            return text.strip()
        except AttributeError:
            return text

    # Try for single frame video first
    # Read CSV detection sequentially
    
    for i in range(45, cctv_range['end'] + 1):
        new_map_image = map_image_filled.copy()
        csv_file_name = str(i).zfill(filename_length)
        csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)

        # Read CSV
        objects = pd.read_csv(csv_path)
        print()
        print("OBJECTS:")
        print(objects)
        print()

        # Eliminate non-vehicles object
        def eliminate_non_vehicle(series):
            detections = series['object_probs'].split(' ')
            for dtc in detections:
                if dtc in coco_vehicle_labels:
                    return True
            return False
        
        objects = objects[objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)]

        # Change string columns into numeric
        # -- Yeah, a little mistake I did when outputting CSV in darknet :( 
        #cols = ['left', 'right', 'top', 'bottom']    
        #for j in cols:
        #    objects[j] = pd.to_numeric(objects[j])

        # For every bounding box vehicles, find the bounding box center point    
        centers = objects.apply(lambda r: [
                          (r['left'] + r['right'])/2,
                          (r['top'] + r['bottom'])/2
                          ], axis=1)
        print("CENTERS:", str(centers).replace("\n", "").replace("  ", " "))

        # Transformation the points
        points_np = np.array([i for i in centers])
        points_transform = np.int32(warpPerspectivePoints(points_np, M))
        print("CENTER TRANSFORMED", str(points_transform).replace("\n", ""))

        # Map the points to the image
        for i in points_transform:
            cv2.circle(new_map_image, tuple(i), 9, (0,255,0), -1)
        cv2.imshow('Image2', overlay_transform)

        cv2.imshow('Image', new_map_image)
        while True:
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        break
finally:
    cv2.destroyAllWindows()

VIEW: patungkuda utara
PTS 1: [[510. 170.] [668. 168.] [664. 437.] [  0. 390.]]
PTS 2: [[648. 167.] [748. 239.] [598. 437.] [497. 362.]]


NameError: name 'map_image_filled' is not defined

In [34]:
# Now let's try it for all video frames.
try:
    rctv = region_cctv_views[1]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)
    polygons = get_points(region_coords, rctv)

    pts1 = polygons['1']
    pts2 = polygons['3']

    # Get transformation matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    pts_transform = np.int32(warpPerspectivePoints(pts1, M))
    
    # Draw polygon containing the transformed region
    overlay_transform = np.uint8(np.ones(map_image.shape)) * 255
    map_image_filled = map_image.copy()
    cv2.fillPoly(overlay_transform, [np.int32(pts_transform)], (0, 0, 255))

    # Get file name length (it is assumed to be same for all the video frames)
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    filename_length = len(file_ext_first_file[0])

    coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

    def strip(text):
        try:
            return text.strip()
        except AttributeError:
            return text

    # Read CSV detection sequentially    
    for i in range(cctv_range['start'], cctv_range['end'] + 1):
        print('\r' + str(i), end='')
        new_map_image = map_image_filled.copy()
        csv_file_name = str(i).zfill(filename_length)
        csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)
        
        # Read CSV
        objects = pd.read_csv(csv_path)
        
        # Do not progress if nothing is detected
        n_detection = objects.shape[0]
        if (n_detection > 0):
            
            # Eliminate non-vehicles object
            def eliminate_non_vehicle(series):
                detections = series['object_probs'].split(' ')
                for dtc in detections:
                    if dtc in coco_vehicle_labels:
                        return True
                return False

            objects = objects[objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)]
            
        n_detection = objects.shape[0]
        if (n_detection > 0):
            # For every bounding box vehicles, find the bounding box center point    
            centers = objects.apply(lambda r: [
                              (r['left'] + r['right'])/2,
                              (r['top'] + r['bottom'])/2
                              ], axis=1)

            # Transformation the points
            points_np = np.array([i for i in centers])
            points_transform = np.int32(warpPerspectivePoints(points_np, M))
            
            # Map the points to the image
            for i in points_transform:
                cv2.circle(new_map_image, tuple(i), 9, (0,255,0), -1)

        cv2.imshow('Image', new_map_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
finally:
    cv2.destroyAllWindows()

64

In [38]:
rgbimg = cv2.cvtColor(np.float64([0, 0, 100]), cv2.COLOR_HSV2RGB)

error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\color.cpp:11109: error: (-215) depth == 0 || depth == 2 || depth == 5 in function cv::cvtColor


In [54]:
cv2.cvtColor(new_map_image, cv2.COLOR_BGR2HSV)

array([[[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       ...,

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]]

In [83]:
for i in range(0, 360):
    print('\r', i, np.uint8(cv2.cvtColor(np.float32([[[i, 1, 1]]]), cv2.COLOR_HSV2RGB)[0, 0] * 255), end='           ')
    time.sleep(0.05)

 56 [255 238   0]           

KeyboardInterrupt: 

In [134]:
type(np.uint8([[new_map_image[0,0]]]).tolist()[0][0])

list

In [72]:
np.uint8([[[360,255,0 ]]])

array([[[104, 255,   0]]], dtype=uint8)